In [358]:
!pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter

use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)






[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\jain9\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [359]:
class NetInterval:
    def __init__(self, lower, upper):
        self._l  = lower
        self._u  = upper
    def affine(self, weight, bias):
        #bias = bias.reshape((1 , bias.shape[0])).expand((self._l.shape[0], -1))
        c = (self._l + self._u) / 2.
        r = (self._u - self._l) / 2.
        r = torch.matmul(r, torch.abs(weight.T))
        c = torch.matmul(c, weight.T)
        for i in range(c.shape[0]):
            c[i] += bias
        return NetInterval(c - r, c + r)
    def relu(self):
        return NetInterval(F.relu(self._l), F.relu(self._u))
    @property
    def l(self):
        return self._l
    @property
    def u(self):
        return self._u

In [360]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc0 = nn.Linear(28*28, 50)
        self.fc1 = nn.Linear(50, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 50)
        self.fc4 = nn.Linear(50,10)

    def forward(self, x):
        x = x.view((-1, 28*28))
        x = F.relu(self.fc0(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
    def output_bounds(self, bounds):
        bounds = bounds.affine(self.fc0.state_dict()['weight'], self.fc0.state_dict()['bias'])
        bounds = bounds.relu()
        bounds = bounds.affine(self.fc1.state_dict()['weight'], self.fc1.state_dict()['bias'])
        bounds = bounds.relu()
        bounds = bounds.affine(self.fc2.state_dict()['weight'], self.fc2.state_dict()['bias'])
        bounds = bounds.relu()
        bounds = bounds.affine(self.fc3.state_dict()['weight'], self.fc3.state_dict()['bias'])
        bounds = bounds.relu()
        bounds = bounds.affine(self.fc4.state_dict()['weight'], self.fc4.state_dict()['bias'])
        return bounds
model = Net()
model = model.to(device)
model.train()

Net(
  (fc0): Linear(in_features=784, out_features=50, bias=True)
  (fc1): Linear(in_features=50, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
)

In [361]:
def evaluate_model_robustness(x, y):
    acc = np.zeros(10)
    for i in range(10):
        eps = .01*(i+1)
        l = torch.reshape(x - eps, (x.shape[0], 28*28))
        u = torch.reshape(x + eps, (x.shape[0], 28*28))
        input_bounds = NetInterval(l, u)
        output = model.output_bounds(input_bounds)
        output.u[:, y] = output.l[:, y]
        acc[i] = torch.argmax(output.u, dim=1).eq(y).sum().item()
    return acc


def train_model_robustness(x, y, eps):
    l = torch.reshape(x - eps, (x.shape[0], 28*28))
    u = torch.reshape(x + eps, (x.shape[0], 28*28))
    input_bounds = NetInterval(l, u)
    output = model.output_bounds(input_bounds)
    output.u[:, y] = output.l[:, y]
    return output.u

In [362]:
def train_model(model, num_epochs):
    learning_rate = 0.0001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()

    tot_steps = 0

    for epoch in range(1,num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            tot_steps += 1
            opt.zero_grad()
            out = model(x_batch)
            batch_loss = ce_loss(out, y_batch)
            batch_loss.backward()
            opt.step()

        tot_test, tot_acc, tot_rob = 0.0, 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_rob += evaluate_model_robustness(x_batch, y_batch)
            tot_test += x_batch.size()[0]
        t2 = time.time()

        print('Epoch %d: Accuracy %.5lf[%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))
        print(tot_rob/tot_test)

In [363]:
train_model(model, 20)

In [364]:
def train_model_robust(model, num_epochs):
    learning_rate = 0.0001

    opt = optim.Adam(params=model.parameters(), lr=learning_rate)

    ce_loss = torch.nn.CrossEntropyLoss()
    tot_steps = 0
    eps = .01
    k = 1.0
    for epoch in range(1,  num_epochs+1):
        t1 = time.time()
        for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            tot_steps += 1
            opt.zero_grad(set_to_none=True)
            out = model(x_batch)
            out2 = train_model_robustness(x_batch, y_batch, eps)
            loss = k*ce_loss(out, y_batch) + (1-k)*ce_loss(out2, y_batch)
            loss.backward()
            opt.step()

        tot_test, tot_acc, tot_rob = 0.0, 0.0, 0.0
        for batch_idx, (x_batch, y_batch) in enumerate(test_loader):
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            out = model(x_batch)
            pred = torch.max(out, dim=1)[1]
            acc = pred.eq(y_batch).sum().item()
            tot_acc += acc
            tot_rob += evaluate_model_robustness(x_batch, y_batch)
            tot_test += x_batch.size()[0]
        t2 = time.time()
        k = max(.5, k-.1)
        print('Epoch %d: Accuracy %.5lf [%.2lf seconds]' % (epoch, tot_acc/tot_test, t2-t1))
        print(tot_rob/tot_test)

In [365]:
model2 = Net()
model2 = model.to(device)
model2.train()
train_model_robust(model2, 20)

Epoch 1: Accuracy 0.85810 [14.80 seconds]
[0.1102 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953]
Epoch 2: Accuracy 0.89350 [14.79 seconds]
[0.1216 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953]
Epoch 3: Accuracy 0.90520 [14.27 seconds]
[0.1696 0.0965 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953 0.0953]
Epoch 4: Accuracy 0.91490 [13.98 seconds]
[0.2677 0.1777 0.1505 0.1259 0.1072 0.0996 0.0966 0.0957 0.0953 0.0953]
Epoch 5: Accuracy 0.92050 [14.11 seconds]
[0.3157 0.1528 0.1048 0.1013 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012]
Epoch 6: Accuracy 0.92570 [13.99 seconds]
[0.2885 0.128  0.1029 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012]
Epoch 7: Accuracy 0.92990 [14.09 seconds]
[0.2568 0.1175 0.1015 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012]
Epoch 8: Accuracy 0.93510 [14.01 seconds]
[0.2557 0.1173 0.1014 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012 0.1012]
Epoch 9: Accuracy 0.93750 [14.54 seconds]
[0.229  0.109  0.1013 0.1012 0.1012 0.1012 0.1